In [0]:
# Databricks notebook source
def path_exists(path):
  try:
    dbutils.fs.ls(path)
    return True
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise

# COMMAND ----------

def download_dataset(source, target):
    files = dbutils.fs.ls(source)

    for f in files:
        source_path = f"{source}/{f.name}"
        target_path = f"{target}/{f.name}"
        if not path_exists(target_path):
            print(f"Copying {f.name} ...")
            dbutils.fs.cp(source_path, target_path, True)

# COMMAND ----------

data_source_uri = "s3://dalhussein-courses/datasets/bookstore/v1/"
dataset_bookstore = 'dbfs:/mnt/demo-datasets/bookstore'
data_catalog = 'hive_metastore'
spark.conf.set(f"dataset.bookstore", dataset_bookstore)

# COMMAND ----------

def get_index(dir):
    files = dbutils.fs.ls(dir)
    index = 0
    if files:
        file = max(files).name
        index = int(file.rsplit('.', maxsplit=1)[0])
    return index+1

# COMMAND ----------

def set_current_catalog(catalog_name):
    spark.sql(f"USE CATALOG {catalog_name}")

# COMMAND ----------

# Structured Streaming
streaming_dir = f"{dataset_bookstore}/orders-streaming"
raw_dir = f"{dataset_bookstore}/orders-raw"

def load_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.parquet"
    print(f"Loading {latest_file} file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_dir}/{latest_file}", f"{raw_dir}/{latest_file}")

    
def load_new_data(all=False):
    index = get_index(raw_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_file(index)
            index += 1
    else:
        load_file(index)
        index += 1

# COMMAND ----------

# DLT
streaming_orders_dir = f"{dataset_bookstore}/orders-json-streaming"
streaming_books_dir = f"{dataset_bookstore}/books-streaming"

raw_orders_dir = f"{dataset_bookstore}/orders-json-raw"
raw_books_dir = f"{dataset_bookstore}/books-cdc"

def load_json_file(current_index):
    latest_file = f"{str(current_index).zfill(2)}.json"
    print(f"Loading {latest_file} orders file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_orders_dir}/{latest_file}", f"{raw_orders_dir}/{latest_file}")
    print(f"Loading {latest_file} books file to the bookstore dataset")
    dbutils.fs.cp(f"{streaming_books_dir}/{latest_file}", f"{raw_books_dir}/{latest_file}")

    
def load_new_json_data(all=False):
    index = get_index(raw_orders_dir)
    if index >= 10:
        print("No more data to load\n")

    elif all == True:
        while index <= 10:
            load_json_file(index)
            index += 1
    else:
        load_json_file(index)
        index += 1

# COMMAND ----------

download_dataset(data_source_uri, dataset_bookstore)
set_current_catalog(data_catalog)

#METHODS OF WRITING IN A TABLE

###1. CREATE OR REPLACE (CRAS)

In [0]:
%sql
CREATE OR REPLACE TABLE orders AS 
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders/export_001.parquet`

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE HISTORY orders ;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2025-01-02T11:01:50.000+0000,1135498371366187,intuztrainee@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1587427143463976),0102-084153-xjuzlqfb,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 22279)",null,Databricks-Runtime/12.2.x-scala2.12


###2. INSERT OVERWRITE

In [0]:
%sql
INSERT OVERWRITE orders
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders/export_001.parquet`;


num_affected_rows,num_inserted_rows
1000,1000


In [0]:
%sql
DESCRIBE HISTORY orders;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-01-02T11:13:15.000+0000,1135498371366187,intuztrainee@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1587427143463976),0102-084153-xjuzlqfb,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 22279)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-01-02T11:01:50.000+0000,1135498371366187,intuztrainee@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1587427143463976),0102-084153-xjuzlqfb,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1000, numOutputBytes -> 22279)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
INSERT OVERWRITE orders
SELECT *,current_timestamp() FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders/export_001.parquet`;
-- This code gives schema mismatch error which conveys that INSERT OVREWRITE DOES'NT OVERWRITE THE ORIGINAL SCHEMA OF TABLE 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1587427143463991>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1587427143463991>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("SU5TRVJUIE9WRVJXUklURSBvcmRlcnMKU0VMRUNUICosY3VycmVudF90aW1lc3RhbXAoKSBGUk9NIHBhcnF1ZXQuYGRiZnM6L21udC9kZW1vLWRhdGFzZXRzL2Jvb2tzdG9yZS9vcmRlcnMvZXhwb3J0XzAwMS5wYXJxdWV0YA==").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50 

###Appending records into table


### 1. INSERT INTO

In [0]:
%sql
SELECT COUNT(*) FROM orders;

count(1)
1000


In [0]:
%sql
INSERT INTO orders 
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders-new/export_004.parquet`; 

num_affected_rows,num_inserted_rows
700,700


In [0]:
%sql
SELECT COUNT(*) FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders-new/export_004.parquet`;

count(1)
700


In [0]:
%sql
SELECT COUNT(*) FROM orders; -- counting the records of orders to check whether the records are successfully appended or not
-- 700 new records + 1000 previous records = 1700 total records

count(1)
1700


In [0]:
%sql
-- insert into does not prevent the inertion of duplicate data 
INSERT INTO orders 
SELECT * FROM parquet.`dbfs:/mnt/demo-datasets/bookstore/orders-new/export_004.parquet`; 
-- if we again insert the same records it will accept them


num_affected_rows,num_inserted_rows
700,700


In [0]:
%sql
-- and now the total count of records in orders = 1000 + 700 + 700 i.e. 2400
SELECT COUNT(*) FROM orders ;

count(1)
2400


To escape from insertion of repeated records we use MERGE INTO 

###MERGE INTO

In [0]:
files = dbutils.fs.ls('/mnt/demo-datasets/bookstore')
display(files)

path,name,size,modificationTime
dbfs:/mnt/demo-datasets/bookstore/books-cdc/,books-cdc/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv/,books-csv/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-csv-new/,books-csv-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/books-streaming/,books-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json/,customers-json/,0,0
dbfs:/mnt/demo-datasets/bookstore/customers-json-new/,customers-json-new/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders/,orders/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-raw/,orders-json-raw/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-json-streaming/,orders-json-streaming/,0,0
dbfs:/mnt/demo-datasets/bookstore/orders-new/,orders-new/,0,0


In [0]:
%sql
-- creating view of customers 
CREATE OR REPLACE TEMP VIEW customer_view AS 
SELECT * FROM json.`${dataset.bookstore}/customers-json`;

-- creating table customers using customer_view
CREATE TABLE customers 
AS SELECT * FROM customer_view;


num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED customers;

col_name,data_type,comment
customer_id,string,null
email,string,null
profile,string,null
updated,string,null
,,
# Detailed Table Information,,
Catalog,hive_metastore,
Database,default,
Table,customers,
Created Time,Fri Jan 03 03:39:59 UTC 2025,


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW updates_view AS
SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json-new/`;

In [0]:
%sql
MERGE INTO customers c
USING  updates_view u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET email = u.email, updated = u.updated
WHEN NOT MATCHED THEN INSERT *;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
301,100,0,201


In [0]:
%sql
DESCRIBE EXTENDED books_csv;

col_name,data_type,comment
book_id,string,null
title,string,null
author,string,null
category,string,null
price,double,null
,,
# Detailed Table Information,,
Catalog,hive_metastore,
Database,default,
Table,books_csv,
